##### use runtime type: TPU or GPU to make it 100 times faster.

## Feature Extractor Transfer Learning
- Transfer learning is of 3 types:<br>
 1. Pre-trained model
 2. Feature Extractor
 3. Fine Tuning

 
 - This file has fine tuning technique.

In [4]:
!wget https://www.dropbox.com/sh/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca?dl=0

--2022-07-31 14:02:55--  https://www.dropbox.com/sh/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca [following]
--2022-07-31 14:02:55--  https://www.dropbox.com/sh/raw/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc74b03ba478af73434309e69eda.dl.dropboxusercontent.com/zip_download_get/BNjifWd88qRRxPzaYrMlKfi0JuKni7i6OGAHPVcfYEuVmw_f4BidaKeUgFHgRZDPFlxWPso8zpEQVpdPaR2w5xp3Oo159sUg4v91W-7bneQf6A# [following]
--2022-07-31 14:02:56--  https://uc74b03ba478af73434309e69eda.dl.dropboxusercontent.com/zip_download_get/BNjifWd88qRRxPzaYrMlKfi0JuKni7i6OGAHPVcfYEuVmw_f4BidaKeUgFHgRZDPFlxWPso8zpEQV

In [5]:
!unzip AABXTlPlU-1f4L_VdS4wY06ca?dl=0

Archive:  AABXTlPlU-1f4L_VdS4wY06ca?dl=0
mapname:  conversion of  failed
   creating: Images/
   creating: Images/cats/
   creating: Images/dogs/
   creating: Images/horses/
   creating: Images/humans/
 extracting: Images/cats/cat.1.jpg   
 extracting: Images/cats/cat.2.jpg   
 extracting: Images/cats/cat.3.jpg   
 extracting: Images/cats/cat.4.jpg   
 extracting: Images/cats/cat.5.jpg   
 extracting: Images/cats/cat.6.jpg   
 extracting: Images/cats/cat.7.jpg   
 extracting: Images/cats/cat.8.jpg   
 extracting: Images/cats/cat.9.jpg   
 extracting: Images/dogs/dog.1.jpg   
 extracting: Images/dogs/dog.2.jpg   
 extracting: Images/dogs/dog.3.jpg   
 extracting: Images/dogs/dog.4.jpg   
 extracting: Images/dogs/dog.5.jpg   
 extracting: Images/dogs/dog.6.jpg   
 extracting: Images/dogs/dog.7.jpg   
 extracting: Images/dogs/dog.8.jpg   
 extracting: Images/dogs/dog.9.jpg   
 extracting: Images/cats/cat.10.jpg  
 extracting: Images/cats/cat.11.jpg  
 extracting: Images/cats/cat.12.jpg  


- There are 4 different folders showing 4 classes named humans, dogs, cats, horses.

In [6]:
ls

'AABXTlPlU-1f4L_VdS4wY06ca?dl=0'   Images/   sample_data/


In [7]:
import os
folders = os.listdir("./Images")
folders

['cats', 'horses', 'humans', 'dogs']

- Here classes names are words. But ML needs numeric data, so we can create a mapping and convert these to numeric values.

In [8]:
label_dict = {
    "cats" : 0,
    "dogs" : 1,
    "horses" : 2,
    "humans" : 3
}

In [9]:
from keras.preprocessing import image
import numpy as np

In [10]:
image_data = []  # to store all the images 
image_labels = [] # to store all the labels for cats, dogs, humans and horses.

for ix in folders:  # we got from os.listdir(..)
    folder_path = "Images/"+ix
    for im in os.listdir(folder_path): # iterate over all images in every folder 
        img = image.load_img(folder_path+"/"+im, target_size=(224,224))
        img_array = image.img_to_array(img)
        image_data.append(img_array)
        label = label_dict[ix]  # folder_name(ix) is class name.
        image_labels.append(label)

In [11]:
print(len(image_data), len(image_labels))

808 808


In [12]:
import random
data = list(zip(image_data, image_labels))

random.shuffle(data)
image_data, image_labels = zip(*data)
print(image_labels[:5])

(3, 2, 2, 3, 2)


- Now data has been shuffled.

In [13]:
# convert lists to numpy arrays now 
X_train = np.array(image_data)
Y_train = np.array(image_labels)
print(X_train.shape, Y_train.shape)

(808, 224, 224, 3) (808,)


In [14]:
# convert data to one hot vectors
from keras.utils import np_utils

Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape, Y_train.shape)

(808, 224, 224, 3) (808, 4)


In [15]:
# Create Resnet-50 model now
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from keras.layers import *
from keras.models import Model

import matplotlib.pyplot as plt

In [16]:
model = ResNet50(include_top=False, weights="imagenet", input_shape=(224,224,3))
model.summary()

94781440/94765736 [==============================] - 2s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [17]:
av1=GlobalAveragePooling2D()(model.output)  # here we pass, from where input 
# should come to this layer
fc1=Dense(256, activation='relu')(av1) # input should come from av1
# no. of neurons in 1st dense layer = 256 and activation='relu'
d1=Dropout(0.5)(fc1) # input should come from fc1 layer
# dropout is by 0.5
fc2=Dense(4, activation='softmax')(d1) # input comes from d1 layer


new_model = Model(inputs=model.input, outputs=fc2)

In [18]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [19]:
adam = Adam(learning_rate=0.00003) # keep learning rate v.small. Reason in notes
new_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [20]:
# let us print all the layers with their indices
for i in range(len(new_model.layers)):
  print(i,new_model.layers[i])

0 <keras.engine.input_layer.InputLayer object at 0x7fa90b96ab90>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7fa904e1dd90>
2 <keras.layers.convolutional.Conv2D object at 0x7fa904e3d990>
3 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fa9045b0fd0>
4 <keras.layers.core.activation.Activation object at 0x7fa9045c76d0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7fa8f05d1790>
6 <keras.layers.pooling.MaxPooling2D object at 0x7fa8f05e2890>
7 <keras.layers.convolutional.Conv2D object at 0x7fa8f05e2310>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fa8f05823d0>
9 <keras.layers.core.activation.Activation object at 0x7fa8f058aa50>
10 <keras.layers.convolutional.Conv2D object at 0x7fa8f0581590>
11 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7fa9045b0c50>
12 <keras.layers.core.activation.Activation object at 0x7fa8f05a0590>
13 <keras.layers.convolutional.Conv2D object at 

In [21]:
for i in range(169):  # make 1st 168 layers non trainable
  new_model.layers[i].trainable = False

new_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

- Here no. of params = 24 million. But trainable params = 1.5 milion only which are from 170-178 layers.

In [22]:
hist = new_model.fit(X_train, Y_train,
                     shuffle=True, # dhuffle data in each epoch
                     batch_size=16,
                     epochs=10,
                     validation_split=0.20)

Epoch 1/10
41/41 [==============================] - 23s 112ms/step - loss: 1.3241 - accuracy: 0.4659 - val_loss: 0.5712 - val_accuracy: 0.8025
Epoch 2/10
41/41 [==============================] - 3s 67ms/step - loss: 0.5672 - accuracy: 0.7957 - val_loss: 0.3385 - val_accuracy: 0.8951
Epoch 3/10
41/41 [==============================] - 3s 67ms/step - loss: 0.3359 - accuracy: 0.8916 - val_loss: 0.2538 - val_accuracy: 0.9074
Epoch 4/10
41/41 [==============================] - 3s 71ms/step - loss: 0.2499 - accuracy: 0.9195 - val_loss: 0.2096 - val_accuracy: 0.9321
Epoch 5/10
41/41 [==============================] - 3s 71ms/step - loss: 0.2003 - accuracy: 0.9536 - val_loss: 0.1806 - val_accuracy: 0.9383
Epoch 6/10
41/41 [==============================] - 3s 67ms/step - loss: 0.1715 - accuracy: 0.9536 - val_loss: 0.1703 - val_accuracy: 0.9506
Epoch 7/10
41/41 [==============================] - 3s 71ms/step - loss: 0.1313 - accuracy: 0.9644 - val_loss: 0.1523 - val_accuracy: 0.9630
Epoch 8/10
